In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor

from sklearn.impute import KNNImputer

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

ModuleNotFoundError: No module named 'pandas'

Adding the data

In [2]:
trainData = pd.read_csv('../input/spaceship-titanic/train.csv')
testData = pd.read_csv('../input/spaceship-titanic/test.csv')
combine = [trainData, testData]

trainData.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
trainData.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [4]:
trainData.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Drop features we dont want to use

In [5]:
trainData = trainData.drop(['Name','Ticket','Cabin','PassengerId'], axis=1)
testData = testData.drop(['Name','Ticket','Cabin'], axis=1)
combine = [trainData, testData]

In [6]:
trainData['Sex'] = trainData['Sex'].map({'male':0, 'female':1}).astype(int)
testData['Sex'] = testData['Sex'].map({'male':0, 'female':1}).astype(int)
combine = [trainData, testData]

trainData.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,22.0,1,0,7.2500,S
1,1,1,1,38.0,1,0,71.2833,C
2,1,3,1,26.0,0,0,7.9250,S
3,1,1,1,35.0,1,0,53.1000,S
4,0,3,0,35.0,0,0,8.0500,S


In [7]:
trainData.Embarked.mode()

0    S
dtype: object

In [8]:
trainData['Embarked'] = trainData['Embarked'].fillna('S')
testData['Embarked'] = testData['Embarked'].fillna('S')


trainData['Embarked'] = trainData['Embarked'].map({'C':0, 'Q':1, 'S':2}).astype(int)
testData['Embarked'] = testData['Embarked'].map({'C':0, 'Q':1, 'S':2}).astype(int)
combine = [trainData, testData]

trainData.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,22.0,1,0,7.2500,2
1,1,1,1,38.0,1,0,71.2833,0
2,1,3,1,26.0,0,0,7.9250,2
3,1,1,1,35.0,1,0,53.1000,2
4,0,3,0,35.0,0,0,8.0500,2


In [9]:
trainData['famSize'] = trainData['SibSp'] + trainData['Parch'] + 1
testData['famSize'] = testData['SibSp'] + testData['Parch'] + 1

trainData.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,famSize
0,0,3,0,22.0,1,0,7.2500,2,2
1,1,1,1,38.0,1,0,71.2833,0,2
2,1,3,1,26.0,0,0,7.9250,2,1
3,1,1,1,35.0,1,0,53.1000,2,2
4,0,3,0,35.0,0,0,8.0500,2,1


Using a KNNImputer to fill missing values of age from Fare and Sibsp features

In [10]:
knnimputer = KNNImputer()

impTrainData = knnimputer.fit_transform(trainData[['Fare','SibSp','Age']])
impTestData = knnimputer.fit_transform(testData[['Fare','SibSp','Age']])

trainData['Age'] = impTrainData[:,2]
testData['Age'] = impTestData[:,2]

Normalizing the Fare feature

In [11]:
testData['Fare'].fillna(value=testData.Fare.mean(),inplace=True)
trainData['Fare'] = trainData['Fare']/trainData['Fare'].abs().max()
testData['Fare'] = testData['Fare']/testData['Fare'].abs().max()
trainData.describe()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,famSize
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,0.352413,30.122678,0.523008,0.381594,0.062858,1.536476,1.904602
std,0.486592,0.836071,0.477990,13.410387,1.102743,0.806057,0.096995,0.791503,1.613459
min,0.000000,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,2.000000,0.000000,22.000000,0.000000,0.000000,0.015440,1.000000,1.000000
50%,0.000000,3.000000,0.000000,29.000000,0.000000,0.000000,0.028213,2.000000,1.000000
75%,1.000000,3.000000,1.000000,38.000000,1.000000,0.000000,0.060508,2.000000,2.000000
max,1.000000,3.000000,1.000000,80.000000,8.000000,6.000000,1.000000,2.000000,11.000000


In [12]:
testData['Age'].fillna(value=testData.Age.mean(),inplace=True)
trainData['Age'] = trainData['Age']/trainData['Age'].abs().max()
testData['Age'] = testData['Age']/testData['Age'].abs().max()
trainData.describe()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,famSize
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,0.352413,0.376533,0.523008,0.381594,0.062858,1.536476,1.904602
std,0.486592,0.836071,0.477990,0.167630,1.102743,0.806057,0.096995,0.791503,1.613459
min,0.000000,1.000000,0.000000,0.005250,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,2.000000,0.000000,0.275000,0.000000,0.000000,0.015440,1.000000,1.000000
50%,0.000000,3.000000,0.000000,0.362500,0.000000,0.000000,0.028213,2.000000,1.000000
75%,1.000000,3.000000,1.000000,0.475000,1.000000,0.000000,0.060508,2.000000,2.000000
max,1.000000,3.000000,1.000000,1.000000,8.000000,6.000000,1.000000,2.000000,11.000000


In [13]:
trainData[['famSize', 'Survived']].groupby(['famSize'], as_index=False).mean().sort_values(by='famSize', ascending=False)

,famSize,Survived
8,11,0.000000
7,8,0.000000
6,7,0.333333
5,6,0.136364
4,5,0.200000
3,4,0.724138
2,3,0.578431
1,2,0.552795
0,1,0.303538


In [14]:
combine = [trainData,testData]
for dataset in combine:
    dataset.loc[ dataset['famSize'] <= 1, 'famSize'] = 0
    dataset.loc[(dataset['famSize'] > 1) & (dataset['famSize'] <= 4), 'famSize'] = 1
    dataset.loc[(dataset['famSize'] > 4), 'famSize']   = 2
    dataset['famSize'] = dataset['famSize'].astype(int)

combine = [trainData, testData]

trainData.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,famSize
0,0,3,0,0.2750,1,0,0.014151,2,1
1,1,1,1,0.4750,1,0,0.139136,0,1
2,1,3,1,0.3250,0,0,0.015469,2,0
3,1,1,1,0.4375,1,0,0.103644,2,1
4,0,3,0,0.4375,0,0,0.015713,2,0


In [15]:
X_train = trainData.drop("Survived", axis=1)
Y_train = trainData["Survived"]
X_test  = testData.drop("PassengerId", axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape

((891, 8), (891,), (418, 8))

In [16]:
# Support Vector Machines

svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, Y_train) * 100, 2)
acc_svc


80.92

This cell finds the best number of neighbors to use for our model

In [17]:
parameters = {"n_neighbors": range(1, 50)}
gridsearch = GridSearchCV(KNeighborsClassifier(), parameters)
gridsearch.fit(X_train, Y_train)

gridsearch.best_params_

{'n_neighbors': 14}

In [18]:
knn = KNeighborsClassifier(n_neighbors = 14)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
knn.score(X_train, Y_train)

0.8170594837261503

In [19]:
# Decision Tree

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
acc_decision_tree

98.2

In [20]:
#parameters = {"n_estimators":[1000,2500, 5000] ,"max_depth": [3,5,7,8,10], "min_samples_split": [5,8,10]}
#gridsearch = GridSearchCV(RandomForestClassifier(), parameters)
#gridsearch.fit(X_train, Y_train)

#gridsearch.best_params_
#{'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 1000}

In [21]:
# Random Forest

random_forest = RandomForestClassifier(n_estimators=5000, min_samples_split = 10, max_depth=5)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest

85.41

In [22]:
gbm = xgb.XGBClassifier(learning_rate = .15, max_depth=7,subsample = .7)
gbm.fit(X_train, Y_train)
gmbScore = round(gbm.score(X_train, Y_train) *100, 2)
gmbScore

94.84

In [23]:
models = pd.DataFrame({
    'Model': [ 'KNN','Random Forest','Decision Tree', 'XGB'],
    'Score': [acc_knn,acc_random_forest,acc_decision_tree,gmbScore]})
models.sort_values(by='Score', ascending=False)

,Model,Score
2,Decision Tree,98.20
3,XGB,94.84
1,Random Forest,85.41
0,KNN,81.71


In [24]:
Y_pred = random_forest.predict(X_test)

In [25]:
submission = pd.DataFrame({
        "PassengerId": testData["PassengerId"],
        "Survived": Y_pred
    })

submission.to_csv('/kaggle/working/submission.csv', index=False)